# Survey Validation Data Challenge - Summer 2019

### Notebook II: BGS GAMA field run.

The purpose of this notebook is to simulate DESI-like spectra of BGS targets bootstrapped from an input set of GAMA targets and redshifts, and then to run the spectra through the full DESI pipeline.
See  https://desi.lbl.gov/trac/wiki/BrightGalaxyWG/SVPlan

* Successful run on 18.12 up to spectro db production -- module swap fiberassign/1.0.0 required for assignment, which doesn't work beyond 18.12 (i.e. 19.2 and master).  Fiberassign dislikes sv_desi_target aswell.

* --survey sv1 is good for select_mock_targets.

* Stuck at 'rawdata' ready state in spectro database - preventing preproc and other dependencies running.

* Initial config. of desisurvey to point to sv tiles file; necessary for add_calibration exposures. 

* Got surveysim up and running by creating single pass, "LO" only groups in rules-sv.yaml

* Correct format for mock-targets.yaml?  E.g. list of contaminants / targets.  See also https://github.com/desihub/fiberassign/issues/198

* As per https://github.com/desihub/survey-validation/blob/master/sv-bgs-decals/main.py, set NUMOBS_MORE=4 in .mtl prior to assignment and specified calls to --nstarpetal 20  --nskypetal 80.

* See for truth.fits warnings:  /global/cscratch1/sd/mjwilson/svdc-summer2018/targets/join_mock_targets.out

*  BGS target selection?  SV bits?  

*  ERROR:run_target_qa:37:<module>: The weightmap file was not passed so systematics cannot be tracked. 
   Try again sending --nosystematics.
    
*  Add Bright star mask and brick boundaries. 

In [ ]:
import desitarget
import os
import sys
import pdb
import time
import pylab as pl
import subprocess
import glob
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
import fitsio
import astropy.io.fits   as fits

from   astropy.table     import Table

In [ ]:
import seaborn as sns

sns.set(style='ticks', font_scale=1.6, palette='Set2')

In [ ]:
%matplotlib inline

In [ ]:
print("Starting at {}".format(time.asctime()))

notebook_start_time = time.time()

In [ ]:
#- Some of these are imported just to establish versions
#- but aren't used in the notebook itself
import desisim
import desispec.io
from   desispec.scripts import pipe
import desitarget.io
import desimodel.io
import desimodel.footprint
import desisurvey
import surveysim
import specsim
from   surveysim.util import add_calibration_exposures
import desiutil.depend
import specter
import redrock
import simqso

from   desiutil import depend


##  Define versions: in flux. 
print(os.environ['DESIMODULES'])

deps = dict()

depend.add_dependencies(deps)
depend.setdep(deps, 'simqso', simqso.__version__)

for p in ('dust', 'desimodules'):
    if p.upper() + '_VERSION' in os.environ:
        depend.setdep(deps, p, os.environ[p.upper() + '_VERSION'])

for codename, version in depend.iterdep(deps):
    print('  {:12s} {}'.format(codename, version))

## Define various directories and other io.

Define and create directories under `$DESI_ROOT/datachallenge/svdc-summer2018` and set environment variables for this mapping:

| Directory             | NB variable   | Environment Variable                              |
|-----------------------|---------------|---------------------------------------------------|
| survey/               | surveydir     | \$DESISURVEY_OUTPUT                               |
| targets/              | targetdir     |                                                   |
| fiberassign/          | fibassigndir  |                                                   |
| spectro/redux/mini/   | reduxdir      | \$DESI_SPECTRO_REDUX/\$SPECPROD                   |
| spectro/sim/mini/     | simdatadir    | \$DESI_SPECTRO_DATA = \$DESI_SPECTRO_SIM/$PIXPROD |

In [ ]:
if True:
    codedir = os.path.join(os.getenv('HOME'), 'desi', 'survey-validation', 'svdc-summer2018')

    #  Large file output. 
    basedir = os.path.join(os.getenv('SCRATCH'), 'svdc-summer2018', 'master')
    
else:
    codedir = os.path.join(os.getenv('DESI_PRODUCT_ROOT'), 'survey-validation', 'svdc-summer2018')
    basedir = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', 'svdc-summer2018')

In [ ]:
print(codedir)
print(basedir)

In [ ]:
is_sv                            = 'sv-'  ## ['sv-']

surveydir                        = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', 'surveysim2017', 'depth_0m')  
targetdir                        = os.path.join(basedir, is_sv + 'targets')

fibassigndir                     = os.path.join(basedir, is_sv + 'fiberassign')

os.environ['DESISURVEY_OUTPUT']  = surveydir

os.environ['DESI_SPECTRO_REDUX'] = os.path.join(basedir, is_sv + 'spectro', 'redux')
os.environ['DESI_SPECTRO_SIM']   = os.path.join(basedir, is_sv + 'spectro', 'sim')

os.environ['PIXPROD']            = 'bgs-gama'
os.environ['SPECPROD']           = 'bgs-gama'

reduxdir                         = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'])
simdatadir                       = os.path.join(os.environ['DESI_SPECTRO_SIM'],   os.environ['PIXPROD'])

os.environ['DESI_SPECTRO_DATA']  = simdatadir

for dd in (surveydir, targetdir, fibassigndir, reduxdir, simdatadir):
    os.makedirs(dd, exist_ok=True)

In [ ]:
samplefile                       = os.path.join(basedir, 'bgs-gama-sample.fits')
surveyconfigfile                 = os.path.join(codedir, 'survey-config.yaml')
surveyrulesfile                  = os.path.join(codedir, 'rules.yaml')

##  Generated by cell 43; overwrite of Dawson tiles to Dark and Bright (program / conditions).
tilesfile                        = os.path.join(basedir, 'tiles', 'bgs-gama-tiles-bright.fits')

#### Specify the random seed for reproducibility of the survey simulations.

In [ ]:
##  Required for surveysim calls (propagates to desisurvey.tiles, see this for further info.).
config                 = desisurvey.config.Configuration()                                                                                                                                       
config.tiles_file.set_value(tilesfile)

In [ ]:
seed                   =   123

In [ ]:
nside_mock_targets     =   64

#### Specify which steps should be redone (only used if the output files already exist).

If any of the following arguments are *True* then all the associated output files and plots will be recreated.  Warning: many of these steps are time-consuming (especially the mock spectra portion of the pipeline).

In [ ]:
overwrite_tiles        =   True
overwrite_surveysim    =   True
overwrite_mock_spectra =   True
overwrite_join_spectra =   True
overwrite_fiberassign  =   True
overwrite_simspec      =   True

## Read the parent sample of GAMA targets.
This parent sample was written by the *bgs-gama-sample.ipynb* notebook.

In [ ]:
def read_gama_sample():
    if os.path.isfile(samplefile):
        gama = Table(fitsio.read(samplefile, ext=1))
        print('Read {} objects from {}'.format(len(gama), samplefile))
        
    else:
        print('Sample file {} not found!'.format(samplefile))
        gama = []
        
    return gama

In [ ]:
gama = read_gama_sample()
gama

## Visualize the tiling of the G02, G09, G12, and G15 GAMA fields. (Remaining G23 field is shallower, 19.2).

Here, we use a tiling solution generated by Kyle Dawson but we change the program name to DARK and set OBSCONDITIONS to zero (=DARK) until the appropriate hooks can be added to *desisurvey*.
G02 was somewhat redefined during the course of the survey. As a result only the part north of −6 deg was observed to high completeness.

In [ ]:
def dawson2dark_tiles():
    """
    Ad hoc changes to the nominal tile file so we can get desisurvey running.
    """
    dawsonfile = os.path.join(basedir, 'tiles', 'bgs-gama-tiles-kdawson.fits')

    if os.path.isfile(dawsonfile):
        '''
        PROGRAM:  'BGS_SV', 
        
        'OBSCONDITIONS':  
        4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4,
        1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4,
        4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1,
        4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4, 1, 4, 4,
        1, 4]
        
        tiles.pass_program

        {0: 'DARK',
         1: 'DARK',
         2: 'DARK',
         3: 'DARK',
         4: 'GRAY',
         5: 'BRIGHT',
         6: 'BRIGHT',
         7: 'BRIGHT'} 
        '''

        tiles = Table(fitsio.read(dawsonfile, ext=1))
        
        print('Updating PROGRAM and OBSCONDITIONS in {}'.format(dawsonfile))
        
        ##  Normal Bright program. 
        tiles['PASS']          =       5
        tiles['PROGRAM']       = 'BRIGHT'
        tiles['OBSCONDITIONS'] =       4
        
        brightfile = basedir + '/tiles/bgs-gama-tiles-bright.fits'
        
        print('Writing {}'.format(brightfile))
        
        tiles.write(brightfile, overwrite=True)


        ##  Normal Dark program.
        tiles['PASS']          = 1
        tiles['PROGRAM']       = 'DARK'
        tiles['OBSCONDITIONS'] = 0
        
        darkfile = basedir + '/tiles/bgs-gama-tiles-dark.fits'
        
        print('Writing {}'.format(darkfile))
        
        tiles.write(darkfile, overwrite=True)
    
    else:
        print('Dawson tiles file {} not found!'.format(dawsonfile))

In [ ]:
def plot_tile(ra, dec, r=1.606, color='k', ax=None):
    '''
    Approximate plot of tile location
    '''
    ang = np.linspace(0, 2*np.pi, 100)
    x   = ra  + r*np.cos(ang)/np.cos(np.radians(dec))
    y   = dec + r*np.sin(ang)

    if ax is None:
        fig, ax = plt.subplots()
    
    ax.plot(x,y, '-', color=color, alpha=0.5)

In [ ]:
def read_gama_tiles(overwrite=False):
    """
    Read the tiles file.
    """
    
    if overwrite:
        dawson2dark_tiles()
    
    if os.path.isfile(tilesfile):
        tiles = Table(fitsio.read(tilesfile, ext=1))
        
        print('Read {} tiles from {}'.format(len(tiles), tilesfile))
        
    else:
        print('Tiles file {} not found!'.format(tilesfile))
        
        tiles = []
    
    return tiles

In [ ]:
def qa_gama_tiles(gama=None, tiles=None, gamafield=None, overwrite=False):
    if gama is None:
        gama  = read_gama_sample()

    if tiles is None:
        tiles = read_gama_tiles()

    fig, ax = plt.subplots(2, 2, figsize=(10, 6))
    ax      =  ax.reshape(4)
    
    for thisax, field in zip(ax, sorted(set(gama['FIELD']))):
        infield = gama['FIELD'] == field
    
        if np.count_nonzero(infield) > 0:
            thisax.scatter(gama['RA'][infield], gama['DEC'][infield], 
                           s=1, alpha=0.5, marker='s')
        
            ww = ( (tiles['RA'] > gama['RA'][infield].min()) * (tiles['RA'] < gama['RA'][infield].max()) * 
                   (tiles['DEC'] > gama['DEC'][infield].min()) * (tiles['DEC'] < gama['DEC'][infield].max()) )
            
            ## tiles[ww].write(basedir + '/tiles/bgs-gama-tiles-%s.fits' % field, overwrite=True)
            
            for tt in tiles[ww]:
                plot_tile(tt['RA'], tt['DEC'], ax=thisax)
            
        thisax.set_xlabel('RA')
        thisax.set_ylabel('Dec')
        
        thisax.invert_xaxis()
        
        thisax.set_title(field)
    
    fig.tight_layout()
    
    if overwrite:
        pngfile = os.path.join(basedir, 'qaplots', 'qa-gama-tiles.png')
        print('Writing {}'.format(pngfile))
        fig.savefig(pngfile)    

In [ ]:
tiles = read_gama_tiles(overwrite=overwrite_tiles)
tiles

In [ ]:
qa_gama_tiles(gama, tiles, overwrite=overwrite_tiles)

In [ ]:
#  Write bgs-gama tile IDs to .txt for fiberassign --surveytiles. 
fname = tilesfile.split('.')[0] + '.txt'

print('Writing %s.' % fname)

np.savetxt(tilesfile.split('.')[0] + '.txt', tiles['TILEID'], fmt='%d')

## Visualize the redshift distribution of the G02, G09, G12, and G15 GAMA fields.

In [ ]:
##  Plot GAMA redshift distribution. 
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

dz      = 0.05 
zbins   = np.arange(0.0, 0.7, dz)

for field in sorted(set(gama['FIELD'])):
    infield   = gama['FIELD'] == field
    counts, _ = np.histogram(gama['Z'][infield], bins=zbins)
    
    pl.plot(zbins[:-1], counts, label=field)
    
ax.set_xlim(0.0, 0.75)

ax.set_xlabel('redshift')
ax.set_ylabel(r'Counts per $\Delta z = %.3lf$' % dz)

ax.legend(ncol=2)    

fig.tight_layout()

## Run survey simulations (successful!  requires rules-sv.yaml)

The code below enables one to simulate SV observations of the GAMA fields using *desisurvey*:  This provides exposure times etc. 
However, since the dates of SV are still uncertain (and three of the four GAMA fields are spring or late-fall fields), we have opted to simulate observing conditions by simply drawing from the full-survey simulations.

In [ ]:
import shutil


##  Change output_path in config file to e.g. /global/cscratch1/sd/mjwilson/svdc-summer2018/survey
configfile      = os.path.join(basedir + '/survey/desisurvey-config.yaml')

if os.path.exists(basedir + '/survey/surveyinit.fits'):
    print('surveyinit.fits already exists; skipping surveyinit')

else:
    logfilename = os.path.join(basedir + '/survey/surveyinit.log')
    cmd         = 'surveyinit --config-file {}'.format(configfile)

    print('Running {}'.format(cmd))
    
    print('Starting at {}; should take ~2 minutes'.format(time.asctime()))
    
    with open(logfilename, 'a') as logfile:
        err = subprocess.call(cmd.split(), stderr=logfile, stdout=logfile)
    
        if err != 0:
            print('surveyinit failed err={}; see {}'.format(err, logfilename))
            raise RuntimeError
        
        else:
            print('Done at {}'.format(time.asctime()))

In [ ]:
##  Error:  No tiles with priority > 0 available.
rulesfile         = os.path.join(basedir + '/survey/rules-sv.yaml')
surveysim_expfile = os.path.join(basedir + '/survey/exposures_surveysim.fits')

if os.path.exists(surveysim_expfile):
    print('Survey sims already done; skipping')

else:
    logfilename = os.path.join(basedir + '/survey/surveysim.log')

    cmd = 'surveysim --config-file {} --rules {} --save-restore'.format(configfile, rulesfile)
    
    print('Running {}'.format(cmd))
    
    print('Starting at {}; should take a few seconds'.format(time.asctime()))
    
    with open(logfilename, 'a') as logfile:
        err = subprocess.call(cmd.split(), stderr=logfile, stdout=logfile)
        
        if err != 0:
            print('surveysim failed err={}; see {}'.format(err, logfilename))
            
            raise RuntimeError
        
        else:
            print('done at {}'.format(time.asctime()))

## Grab SV-like exposures (matching EBV and program) from 2017 survey simulation

In [ ]:
def sample_survey_simulations(tiles, overwrite=False, seed=None):
    '''
    Generate the exposures file by drawing from the 2017 full-survey simulations.
    '''
    import copy
    import desisurvey
    
    from   astropy.table import Column
    
    
    expfile = os.path.join(surveydir, 'exposures.fits')
    
    if overwrite or not os.path.isfile(expfile):        
        from astropy.table  import vstack
        from surveysim.util import add_calibration_exposures

        
        config       = desisurvey.config.Configuration()                                                                                                                                       
        config.tiles_file.set_value(tilesfile)
        
        rand         = np.random.RandomState(seed)
        
        simexpfile   = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', 'surveysim2017',   'depth_0m', 'exposures.fits')
        #simexpfile  = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', 'svdc-summer2018', 'survey',   'exposures.fits')
        #simexpfile  = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', 'surveysim2018',   'survey',   'exposures.fits')
        
        if not os.path.isfile(simexpfile):
            print('Exposures file {} not found!'.format(simexpfile))
            raise IOError

        exp = Table.read(simexpfile)

        print('Read {} parent exposures from {}'.format(len(exp), simexpfile))

        # flag CR splits.  CR split? 
        _, uniq = np.unique(exp['TILEID'].data, return_index=True)
        
        explist = []
        
        ##  Range of EBV for tiles:  0.0504712 0.0192004    
        for tt in tiles:
            # Find exposures with the correct PROGRAM and close in Galactic 
            # reddening and then choose one at random (with uniform probability).
            # If there is more than one exposure, take both (CR split).            
            ww = np.where( (exp['PROGRAM'][uniq] == tt['PROGRAM']) * 
                           (np.abs(exp['EBMV'][uniq]-tt['EBV_MED']) < 0.02) )[0]
            
            if np.count_nonzero(ww) == 0:
                ww = np.where( (exp['PROGRAM'][uniq] == tt['PROGRAM']) * 
                               (np.abs(exp['EBMV'][uniq]-tt['EBV_MED']) < 0.051) )[0]
                
                if np.count_nonzero(ww) == 0:
                    print('Insufficient comparable exposures!')
                    raise ValueError
                
            # choose one at random.
            this = rand.choice(ww)
            
            # bring back CR splits.
            these = exp['TILEID'] == exp['TILEID'][uniq][this]
            
            _explist = vstack(exp[these])
            
            # Replace exposure data with criteria for Kyle's tiles.
            _explist['TILEID'] = tt['TILEID']
            _explist['PASS']   = tt['PASS']
            _explist['EBMV']   = tt['EBV_MED']
            _explist['RA']     = tt['RA']
            _explist['DEC']    = tt['DEC']
            
            explist.append(_explist)

        explist                = vstack(explist)
        
        # Assign all 94 tiles to a single (wonderful) NIGHT and MJD.
        this                   = np.where(exp['NIGHT'].data == '20191201')[0][0]
        explist['NIGHT'][:]    = exp['NIGHT'][this]
        explist['MJD'][:]      = exp['MJD'][this]
        
        # Add monotonically increasing epsilon perturbation to MJD to beat diff(MJD) > 0 catch in surveysim/util.py  
        epsilons               = 1.e-1 * np.sort(np.random.uniform(0., 1., len(explist['MJD'])))
        explist['MJD']        += epsilons
        
        '''
        Deprecated:  instead set desisurvey config.  See desisurvey.tiles
        
        # Save tile ID hack for add_calibration_exposures. 
        _keepTileIDs           = copy.copy(explist['TILEID'].quantity)

        # Rewrite tile IDs to nominal desi for add_calibration_exposures error. 
        desi_tileIDs           = desisurvey.tiles.get_tiles()
        
        explist['TILEID']      = desi_tileIDs.tileID[:len(explist['TILEID'])]
        '''     
        explist                = add_calibration_exposures(explist, flats_per_night=1, arcs_per_night=1)

        # Rewrite nominal desi tile program to bright. 
        explist['PROGRAM'][2:] = 'BRIGHT'
        
        # Reset TileIDs.
        '''
        Deprecated:  instead set desisurvey config.  See desisurvey.tiles
        explist['TILEID']  = Column(data=np.concatenate((np.array([-1, -1]), _keepTileIDs)), name='TILEID')
        '''

        # sequential IDs
        explist['EXPID']       = np.arange(len(explist))
        
    else:
        print('Simulated observing already completed.')
        explist = Table.read(expfile)
        print('Read {} exposures from {}'.format(len(explist), expfile))

    return explist

In [ ]:
#%time explist = survey_simulations(tiles, overwrite=overwrite_surveysim)
%time explist  = sample_survey_simulations(tiles, overwrite=overwrite_surveysim, seed=seed)
explist

####  Alter BGS MOON PARAMETER dependence - hack around survey sim;  Stripped from minitest, but useful?

The default MOONALT, MOONSEP, and MOONFRAC columns are data model placeholders but not filled in with meaningful values. Correct that. 

Notes:
the code below is approximate (but still better than using MOONALT=-10 for BRIGHT exposures) surveysim doesn't currently vary the bright exposure time for lunar parameters (!), so filling this in with actual parameters instead of median defaults is arguably the wrong thing to do, but it does give us a variation of conditions from which one could calculate how the exposure times should vary.

In [ ]:
import desisurvey.ephem
import warnings


warnings.filterwarnings('ignore', message=r'Tried to get polar motions for times after IERS data.')

def add_moon_params(explist):
    '''
    Fills MOONFRAC, MOONALT, MOONSEP columns in explist.
    '''    

    ephem_file = glob.glob(basedir + '/survey/ephem*.fits')[0]
    ephem      = Table.read(ephem_file)
    
    explist['MOONFRAC'] = np.interp(explist['MJD'], ephem['brightdusk'], ephem['moon_illum_frac'])

    ii = np.searchsorted(ephem['brightdusk'], explist['MJD'])-1

    assert np.all(ii>=0)
    
    for i, j in zip(ii, range(len(explist['MJD']))):
        if explist['FLAVOR'][j] != 'science':
            continue
        moon                  = desisurvey.ephem.get_object_interpolator(ephem[i], 'moon', altaz=True)
        mjd                   = explist['MJD'][j]
        explist['MOONALT'][j] = moon(mjd)[0]
        moon_dec, moon_ra     = desisurvey.ephem.get_object_interpolator(ephem[i], 'moon', altaz=False)(mjd)        
        phi1, phi2            = np.radians(moon_dec), np.radians(explist['DEC'][j])
        theta1, theta2        = np.radians(moon_ra), np.radians(explist['RA'][j])
        
        #- Haversine formula
        d                     = 2*np.arcsin(np.sqrt(np.sin(0.5*(phi2-phi1))**2 + np.cos(phi1)*np.cos(phi2)*np.sin(0.5*(theta2-theta1))**2))
        
        explist['MOONSEP'][j] = np.degrees(d)

In [ ]:
##  add_moon_params(explist)

In [ ]:
# Write exposure list. 
explist.write(basedir + '/survey/bgs-gama-exposures.fits', format='fits', overwrite='True')

#### Visualize which healpixels cover the observed tiles.

In [ ]:
def plot_healpix(nside, pixels, ax=None):
    '''
    Plot healpix boundaries; doesn't work at RA wraparound.
    '''
    if ax is None:
        fig, ax    = plt.subplots()
        
    for p in pixels:
        xyz        = hp.boundaries(nside, p, nest=True)
        theta, phi = hp.vec2ang(xyz.T)
        theta      = np.concatenate([theta, theta[0:1]])
        phi        = np.concatenate([phi, phi[0:1]])
        ra, dec    = np.degrees(phi), 90-np.degrees(theta)
        
        ax.plot(ra, dec, '.', color='0.6') 

In [ ]:
def tiles2pixels(tiles, nside=64, fact=2**8):
    import desimodel.footprint
    
    
    pixels = desimodel.footprint.tiles2pix(nside, tiles, fact=fact)
    nexp   = np.count_nonzero(np.in1d(explist['TILEID'], tiles['TILEID']))

    print('{} tiles covered by {} exposures and {} nside={} healpixels'.format(len(tiles), nexp, len(pixels), nside))

    return pixels

In [ ]:
def qa_observed_tiles():
    isbright = explist['PROGRAM'] == 'BRIGHT'
    isgray   = explist['PROGRAM'] == 'GRAY'
    isdark   = explist['PROGRAM'] == 'DARK'
    
    fig, ax  = plt.subplots(figsize=(12, 8))

    ax.plot(explist['RA'][isdark], explist['DEC'][isdark], 'o', color='deepskyblue', ms=10, mew=2, label='Dark exposure', alpha=1.0)
    ax.plot(  tiles['RA'],           tiles['DEC'],         'o', ms=5, color='k', label='Requested tiles', alpha=0.5)
    
    if np.sum(isgray) > 0:
        ax.plot(explist['RA'][isgray], explist['DEC'][isgray], 's', 
                color='0.6', ms=10, label='gray')
    
    if np.sum(isbright) > 0:
        ax.plot(explist['RA'][isbright], explist['DEC'][isbright], 'd', 
                color='m', ms=5, mew=2, label='bright')
    
    ax.legend(loc='upper left')

    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')

In [ ]:
qa_observed_tiles()

In [ ]:
def qa_tiles2pixels(tiles, nside=64, overwrite=False, plotit=False):
  pixels = tiles2pixels(tiles, nside=nside)
  
  if plotit:
    ##  Get science rather than calibration exposures.
    sci    = explist['FLAVOR'] == 'science'
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

    ax1.scatter(tiles['RA'], tiles['DEC'], marker='s', edgecolor='k', facecolor='none', lw=2, label='Tile Centers')
    ax1.scatter(explist['RA'][sci], explist['DEC'][sci], marker='o', alpha=0.7, label='Science exposures', s=6)
    
    ax1.legend(loc='upper left', markerscale=1.5)

    ax1.invert_xaxis()
    ax1.set_ylim(-7.5, 7.5)
    
    ax1.set_xlabel('RA')
    ax1.set_ylabel('Dec')

    plot_healpix(nside, pixels, ax=ax2)
    
    color = dict(DARK='m', GRAY='b', BRIGHT='m')
    
    for program in ['DARK', 'GRAY', 'BRIGHT']:
        ii = (tiles['PROGRAM'] == program)
        ax2.plot(tiles['RA'][ii], tiles['DEC'][ii], '.', color=color[program], alpha=1.)
        jj = tiles['PROGRAM'] == program

        for t in tiles[jj]:
            plot_tile(t['RA'], t['DEC'], color=color[program], ax=ax2)
            
    xlim = ax2.get_xlim()
    
    ax1.set_xlim(xlim)
    ax2.set_xlim(xlim)
      
    fig.tight_layout()
    
    if overwrite:
        pngfile = os.path.join(basedir, 'qaplots', 'qa-gama-tiles-healpixels.png')
        print('Writing {}'.format(pngfile))
        fig.savefig(pngfile)        
    
    return pixels

In [ ]:
pixels = qa_tiles2pixels(tiles, nside=nside_mock_targets, overwrite=overwrite_tiles)
pixels

In [ ]:
#  Plot up histograms of EXPTIME, SEEING, TRANSPARENCY, AIRMASS, MOONSEP, MOONFRAC, MOONALT.
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1, 6, figsize=(36, 6), sharey=True)

ax1.hist(explist['EBMV'], bins=20, color='deepskyblue')
ax1.set_xlabel(r'$E(B-V)$')

ax2.hist(explist['SEEING'], bins=20, color='deepskyblue')
ax2.set_xlabel(r'Seeing [arcseconds]')

ax3.hist(explist['TRANSPARENCY'], bins=20, color='deepskyblue')
ax3.set_xlabel(r'Transparency')

ax4.hist(explist['AIRMASS'], bins=20, color='deepskyblue')
ax4.set_xlabel(r'Airmass')

ax5.hist(explist['MOONSEP'], bins=20, color='deepskyblue')
ax5.set_xlabel(r'Moon separation [deg.]')

ax6.hist(explist['MOONFRAC'], bins=20, color='deepskyblue')
ax6.set_xlabel(r'Moon fraction')

ax1.set_title('SV-like exposures (from 2017 sim.)\n', fontsize=28)

## 
pexp, counts = np.unique(explist['PROGRAM'], return_counts=True)

print('\n\nProgram split:\n')
print(pexp)
print(counts)
print('\n\n')

In [ ]:
#  Plot EXPTIME vs SEEING, TRANSPARENCY, AIRMASS, MOONSEP, MOONFRAC, MOONALT.
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(36, 6), sharey=True)

ax1.plot(explist['EBMV'], explist['EXPTIME'], 'o', color='deepskyblue')
ax1.set_xlabel(r'$E(B-V)$')
ax1.set_ylabel(r'Exposure [s]')

ax2.plot(explist['SEEING'], explist['EXPTIME'], 'o', color='deepskyblue')
ax2.set_xlabel(r'Seeing')
ax2.set_ylabel(r'Exposure [s]')

ax3.plot(explist['AIRMASS'], explist['EXPTIME'], 'o', color='deepskyblue')
ax3.set_xlabel(r'Airmass')
ax3.set_ylabel(r'Exposure [s]')

ax4.plot(explist['MOONSEP'], explist['EXPTIME'], 'o', color='deepskyblue')
ax4.set_xlabel(r'Moon separation [deg]')
ax4.set_ylabel(r'Exposure [s]')

ax5.plot(explist['MOONFRAC'], explist['EXPTIME'], 'o', color='deepskyblue')
ax5.set_xlabel(r'Moon fraction')
ax5.set_ylabel(r'Exposure [s]')

## Simulate noiseless spectra using select_mock_targets.
Then, merge the target, sky, and stdstar catalogs and build the Merged Target List (MTL) file.

In [ ]:
##  From here, process the minimal amount, e.g. one tile. 
fonetile    = basedir + '/tiles/bgs-gama-tiles-100000.fits'

onetile     = Table(fitsio.read(fonetile, ext=1))

##  Pixels for that one tile.
onetile_pix = tiles2pixels(onetile, nside=64)

print(onetile_pix)

In [ ]:
onetile['RA']

In [ ]:
def is_select_mock_targets_done(targetdir, pixels, verbose=True):
    done = True

    for filetype in ['targets', 'truth', 'sky', 'standards-dark', 'standards-bright']:
        filenames = glob.glob(os.path.join(targetdir, '*', '*', '{}*.fits'.format(filetype)))
        
        if verbose:
            print('{}/{} {} files'.format(len(filenames), len(pixels), filetype))

        if len(filenames) != len(pixels):
            done = False
            
    return done

In [ ]:
def select_mock_targets(tilesfile, nside=64, overwrite=False, verbose=False, mpi=True, no_spectra=False):
    """
    Run select_mock_targets.
    
    For debugging in an interactive node:
 
    #  Config mixes standards across both science and standards file.  Currently an issue for fiberassign/master. 
    salloc -N 15 -q interactive -L SCRATCH -C haswell -t 00:30:00 -n 30 -c 16

    #  Won't do anything if the healpix split directories already exist. 

    mpi_select_mock_targets 
       --output_dir /global/cscratch1/sd/mjwilson/svdc-summer2018/targets
       --config /global/homes/m/mjwilson/desi/survey-validation/svdc-summer2018/mock-targets.yaml
       --seed 123 
       --nproc 16 
       --nside 64 
       --healpixels 25912 25913 25914 25915 25916 25917 25918 25919 25962 26000 26001 26002 26003 26004 26005 26006 26007 26012 26013 26048 26050
       --survey sv1       ##  [main, sv1] 
       
    Note:  See cell 36 for necessary pixel list.  E.g. pixels [21883] took 16.6 minutes.
    """
    logfilename = os.path.join(targetdir, 'select_mock_targets.log')
    
    if overwrite or not is_select_mock_targets_done(tilesfile, nside=nside, verbose=verbose):
        print('Starting select_mock_targets at {}'.format(time.asctime()))
        print('Logging to {}'.format(logfilename))

        configfile = os.path.join(codedir, 'mock-targets.yaml')

        if mpi:
            cmd  = "srun -A desi -N 15 -n 30 -c 16 -C haswell -t 00:60:00 --qos interactive"
            cmd += " mpi_select_mock_targets --output_dir {targetdir} --config {configfile}"
            cmd += " --seed {seed} --nproc 16 --nside {nside} --tiles {tilesfile} "
        
        else:
            cmd  = "select_mock_targets --output_dir {targetdir} --config {configfile}"
            cmd += " --seed {seed} --nproc 4 --nside {nside} --tiles {tilesfile} --overwrite "
        
        if no_spectra:
            cmd += "--no-spectra"
        
        cmd = cmd.format(targetdir=targetdir, tilesfile=tilesfile, 
                         configfile=configfile, seed=seed, nside=nside)
        
        print(cmd)
        
        with open(logfilename, 'w') as logfile:
            err = subprocess.call(cmd.split(), stderr=logfile, stdout=logfile)
            
            if err != 0:
                print('select_mock_targets failed err={}; see {}'.format(err, logfilename))
                
            else:
                print('done at {}'.format(time.asctime()))
 
    else:
        print('All done with select_mock_targets; see log file {}'.format(logfilename))            

In [ ]:
##  Pixels [21883] took 16.6 minutes; 21 nside=64 healpixels to be processed. 
##  
##  NOTE:  If output directories exist of given healpixel, no reprocessing will occur.  Delete first.  
select_mock_targets(fonetile, nside=nside_mock_targets, verbose=True, mpi=True, overwrite=overwrite_mock_spectra, no_spectra=True)

In [ ]:
is_select_mock_targets_done(targetdir, onetile_pix, verbose=True)

In [ ]:
def join_targets_truth_mtl(overwrite=False):
    mtlfile     = os.path.join(targetdir, 'mtl.fits')
    truthfile   = os.path.join(targetdir, 'truth.fits')
    targetsfile = os.path.join(targetdir, 'targets.fits')

    if (overwrite or not os.path.isfile(mtlfile) or not os.path.isfile(targetsfile) or not os.path.isfile(truthfile)):        
        cmd = "join_mock_targets --mockdir {} --overwrite".format(targetdir)
        
        print(cmd)
        
        err = subprocess.call(cmd.split())
        
        if err != 0:
            print('join_mock_targets failed err={}'.format(err))
        
        else:
            print('Successfully joined all targets and truth catalogs.')
        
    else:
        print('Using existing truth.fits {}'.format(truthfile))        
        print('Using existing targets.fits file {}'.format(targetsfile))        
        print('Using existing merged target list {}'.format(mtlfile))        

In [ ]:
join_targets_truth_mtl(overwrite=overwrite_join_spectra)

## Do basic sanity check on the result. 

In [ ]:
targets, target_hdr = fitsio.read(os.path.join(targetdir, 'targets.fits'), header=True)
truth               = fitsio.read(os.path.join(targetdir, 'truth.fits'))
mtl                 = fitsio.read(os.path.join(targetdir, 'mtl.fits'))
std, std_hdr        = fitsio.read(os.path.join(targetdir, 'standards-dark.fits'), header=True)
sky                 = fitsio.read(os.path.join(targetdir, 'sky.fits'))

pl.figure(figsize = (6,4))

pl.plot(mtl['RA'], mtl['DEC'], 'b,', alpha=0.1)
pl.plot(std['RA'], std['DEC'], 'm.', alpha=0.5)

pl.xlabel('RA')
pl.ylabel('DEC')

pl.xlim(225,  210)
pl.ylim(-2.5, 4.0)

pl.title('Single G15 exposure.')

## Run target selection QA
Successfully outputs to /global/cscratch1/sd/mjwilson/svdc-summer2018/targets/qa

In [ ]:
targetfile  = os.path.join(targetdir,   'targets.fits')
targetQAdir = os.path.join(targetdir,   'qa')
targetQAlog = os.path.join(targetQAdir, 'target-qa.log')

os.makedirs(targetQAdir, exist_ok=True)

# ERROR:run_target_qa:37:<module>: The weightmap file was not passed so systematics cannot be tracked. 
# Try again sending --nosystematics.
cmd = 'run_target_qa {} {} --mocks --nside 32 --nosystematics'.format(targetfile, targetQAdir)

print(cmd)

with open(targetQAlog, 'w') as logfile:
    err = subprocess.call(cmd.split(), stdout=logfile, stderr=logfile)

if err != 0:
    print('ERROR running {}'.format(cmd))

    msg = 'see {}'.format(targetQAlog)
    
    raise RuntimeError(msg)

In [ ]:
from IPython.display import Image, display

Image(targetQAdir + '/mock-nz-BGS_ANY.png',   width=8e2)

#### Rewrite NUMOBS_MORE in .mtl: 4 observations per target in SV. 

In [ ]:
def set_sv_numobs(numobs=4):
  mtl = fits.open(targetdir + '/mtl.fits')

  mtl[1].data['NUMOBS_MORE'][mtl[1].data['NUMOBS_MORE'] > 0] = numobs

  mtl.writeto(targetdir + '/mtl.fits', overwrite=True)

  print('Successfully rewritten %s/mtl.fits to SV-like NUMOBS_MORE = 4.' % targetdir)

In [ ]:
set_sv_numobs(1)

## Fiber assignment

In [ ]:
def is_fiberassign_done(tilesfile, verbose=False):
    """
    Check whether fiberassign successfully finished.
    """
    
    tiles = Table(fitsio.read(tilesfile, ext=1))
    done  = True
    
    for tileid in tiles['TILEID']:
        tilefile = os.path.join(fibassigndir, 'fiberassign_{:05d}.fits'.format(tileid))
        
        if not os.path.exists(tilefile):
            done = False
            
            if verbose:
                print('Missing {}'.format(tilefile))

    return done

In [ ]:
def run_fiberassign(tilesfile, overwrite=False, verbose=False):
    '''
    To debug:  
    
    salloc -N 15 -q interactive -L SCRATCH -C haswell -t 00:30:00 -n 30 -c 16
    
    module swap fiberassign/1.0.0
    
    fiberassign  --mtl /global/cscratch1/sd/mjwilson/svdc-summer2018/targets/mtl.fits 
                 --stdstar /global/cscratch1/sd/mjwilson/svdc-summer2018/targets/standards-dark.fits 
                 --sky /global/cscratch1/sd/mjwilson/svdc-summer2018/targets/sky.fits 
                 --surveytiles $HOME/desi/survey-validation/svdc-summer2018/bgs-gama-tiles-kdawson-100000.txt 
                 --footprint /global/cscratch1/sd/mjwilson/svdc-summer2018/tiles/bgs-gama-tiles-100000.fits 
                 --fibstatusfile /global/homes/m/mjwilson/desi/survey-validation/svdc-summer2018/fiberstatus.ecsv 
                 --outdir /global/cscratch1/sd/mjwilson/svdc-summer2018/fiberassign 
                 --overwrite
                 
                 To be added:

                 --nstarpetal 20 
                 --nskypetal 80 
                 --overwrite
    '''
    
    logfilename = os.path.join(fibassigndir, 'fiberassign.log')

    if overwrite or not is_fiberassign_done(tilesfile, verbose=verbose):
        print('Generating lists of dark and bright tiles')

        tiles = Table(fitsio.read(tilesfile, ext=1))

        bx, dx = None, None

        for tileid, program  in zip(tiles['TILEID'], tiles['PROGRAM']):
            if program == 'BRIGHT':
                if bx is None:
                    bx = open(os.path.join(fibassigndir, 'bright-tiles.txt'), 'w')
                bx.write(str(tileid)+'\n')
                
            else:
                if dx is None:
                    dx = open(os.path.join(fibassigndir, 'dark-tiles.txt'), 'w')
                dx.write(str(tileid)+'\n')
                
        if bx:
            bx.close()
        
        if dx:
            dx.close()

        # Remove any leftover tile files
        for tilefile in glob.glob(fibassigndir + '/tile_*.fits'):
            os.remove(tilefile)

        cmd   = "fiberassign "
        cmd  += " --mtl {}/mtl.fits".format(targetdir)
        cmd  += " --stdstar {}/{{stdfile}}".format(targetdir)
        cmd  += " --sky {}/sky.fits".format(targetdir)
        cmd  += " --surveytiles {}/{{surveytiles}}".format(fibassigndir)
        cmd  += " --footprint {tilesfile}"
        #cmd += " --positioners {}/data/focalplane/fiberpos.fits".format(os.getenv('DESIMODEL'))
        cmd  += " --fibstatusfile {}/fiberstatus.ecsv".format(codedir)
        cmd  += " --outdir {}".format(fibassigndir)

        # Run fiberassign
        print('Logging to {}'.format(logfilename)); print()
        
        with open(logfilename, 'w') as logfile:
            for program in ['dark', 'bright']:
                stdfile     = 'standards-{}.fits'.format(program)
                surveytiles =      '{}-tiles.txt'.format(program)
        
                if os.path.isfile(os.path.join(fibassigndir, surveytiles)):
                    cmdx = cmd.format(stdfile=stdfile, surveytiles=surveytiles,
                                      tilesfile=tilesfile)
                    print(cmdx)
                
                    err = subprocess.call(cmdx.split(), stdout=logfile, stderr=logfile)

                    if err != 0:
                        print('fiberassign failed err={}; see {}'.format(err, logfilename))
        
        if is_fiberassign_done(tilesfile, verbose=True):
            print('Success; Running QA.')
            print()

            !qa-fiberassign $fibassigndir/tile*.fits --targets $targetdir/targets.fits
        
        else:
            print('ERROR: missing fiberassign output files')
        
    else:
        print('Finished fiber assignment; see log file {}'.format(logfilename))            

In [ ]:
#  Successfully run in shell  --  module swap fiberassign/1.0.0 required ... 
%time run_fiberassign(os.path.join(codedir, 'bgs-gama-tiles-%s.fits' % 'G02'), overwrite=overwrite_fiberassign)

###  Fiber assignment QA

In [ ]:
assignQAdir = os.path.join(basedir, 'fiberassign', 'qa')
assignQAlog = os.path.join(assignQAdir, 'assign-qa.log')

os.makedirs(assignQAdir, exist_ok=True)

cmd     = 'qa-fiberassign --verbose %s/fiberassign/tile-100000.fits --targets %s/targets.fits' % (basedir, targetdir)

print(cmd)

with open(assignQAlog, 'w') as logfile:
  err = subprocess.call(cmd.split(), stdout=logfile, stderr=logfile)

if err != 0:
  print('Fiberassign QA failed err={}; see {}'.format(err, assignQAlog))

### Combine surveysim, mocks, and fiberassign into simspec files.

[This step took roughly 27 minutes on my laptop with two tiles.]

In [ ]:
def is_newexp_done(explist, verbose=False):    
    numnights     = len(set(explist['NIGHT']))
    nexp          = len(explist)  #- 3 arc/night + 3 flat/night + science
    simspecfiles  = glob.glob(simdatadir + '/*/simspec*.fits')
    fibermapfiles = glob.glob(simdatadir + '/*/fibermap*.fits')
    
    if verbose:
        print('{}/{} simspec files'.format(len(simspecfiles), nexp))
        print('{}/{} fibermap files'.format(len(fibermapfiles), nexp))

    if len(simspecfiles) != nexp:
        return False
    
    elif len(fibermapfiles) != nexp:
        return False
    
    else:
        return True

In [ ]:
def run_newexp(tilefile, explist, overwrite=False, mpi=False, dryrun=False):
    '''
    Run newexp.

    Debug:
        
    salloc -N 15 -q interactive -L SCRATCH -C haswell -t 00:30:00 -n 15 -c 32

    srun -A desi -N 15 -n 15 -c 32 -C haswell -t 00:120:00 --qos debug 
    
    wrap-newexp 
         --fiberassign /global/cscratch1/sd/mjwilson/svdc-summer2018/fiberassign 
         --mockdir /global/cscratch1/sd/mjwilson/svdc-summer2018/targets 
         --obslist /global/cscratch1/sd/mjwilson/svdc-summer2018/survey/bgs-gama-exposures.fits 
         --tilefile /global/cscratch1/sd/mjwilson/svdc-summer2018/tiles/bgs-gama-tiles-100000.fits  
         --outdir $SCRATCH/svdc-summer2018/spectro/sim/bgs-gama/ 
         --force
         --mpi
    
    Note this logs directly to projectdirs/user/
    e.g.  /global/project/projectdirs/desi/spectro/sim/mjwilson/20210421/00011517/simspec-00011517.log    

    Output:
    /global/project/projectdirs/desi/spectro/sim/mjwilson/20191201/00000000/simspec-00000000.log

    i.e.  Calibration written to 
    
    /global/project/projectdirs/desi/spectro/sim/mjwilson/20191201/
    

    Science written to outdir. 
    '''
    
    logfilename  = os.path.join(os.environ['SCRATCH'] + '/svdc-summer2018/spectro/sim/bgs-gama/' + 'newexp.log')
    #logfilename = os.path.join(simdatadir, 'newexp.log')
    
    if overwrite or not is_newexp_done(explist):
        logfilename = os.path.join(simdatadir, 'newexp.log')
        
        print('Logging to {}'.format(logfilename))

        cmd    = " wrap-newexp --fiberassign {}".format(fibassigndir)
        cmd   += " --mockdir {}".format(targetdir)
        cmd   += " --obslist {}/survey/bgs-gama-exposures.fits".format(basedir)
        cmd   += " --tilefile {}".format(tilefile)
        cmd   += " --outdir {}/spectro/sim/bgs-gama/".format(basedir)

        if dryrun:
          cmd += " --dryrun"
        
        if overwrite:
            cmd += " --force"
        
        print('Starting at {}'.format(time.asctime()))
        
        if 'NERSC_HOST' in os.environ:
            nodes      = 15
            nersc_cmd  = "srun -A desi -N {nodes} -n {nodes} -c 32".format(nodes=nodes)
            nersc_cmd += " -C haswell -t 00:15:00 --qos interactive"
            cmd        = nersc_cmd + cmd 
            
        if mpi: 
            cmd       += ' --mpi'
        
        print(cmd)

        with open(logfilename, 'w') as logfile:
            err = subprocess.call(cmd.split(), stdout=logfile, stderr=logfile)
            
            if err != 0:
                print('ERROR {} running wrap-newexp; see {}'.format(err, logfilename))
                
            else:
                print('done')            
    else:
        print('newexp is done')
        
        is_newexp_done(explist, verbose=True)

In [ ]:
#  Single tile: 1 arc, 1 flat and 1 science exposure.
%time run_newexp(os.path.join(codedir, 'bgs-gama-tiles-%s.fits' % '100000'), explist, overwrite=overwrite_simspec, mpi=True)

# Single GAMA field.
# %time run_newexp(os.path.join(codedir, 'bgs-gama-tiles_%s.fits' % 'G02'), explist, overwrite=overwrite_simspec, mpi=True)

### Generate noisy uncalibrated spectra  using fastframe.

fastframe is a stripped down version of quickgen, and it uses specsim under the hood. 
specsim is memory hungry so we are limited to one process per node, leaving the other 
cores idle.

6.3 minutes for 12 arc, 12 flat, 18 science exposures on 15 nodes.

In [ ]:
def is_fastframe_done(explist, reduxdir, verbose=False):
    nflat      = np.count_nonzero(explist['FLAVOR'] == 'flat')
    nscience   = np.count_nonzero(explist['FLAVOR'] == 'science')
    nframe     = 30 * (nflat + nscience)
    
    framefiles = glob.glob(reduxdir+'/exposures/*/*/frame*.fits')

    if verbose:
        print('{}/{} frame files'.format(len(framefiles), nframe))
    
    if len(framefiles) != nframe:
        return  False
    
    else:
        return  True

In [ ]:
def run_fastframe(overwrite=False):
    '''
    Debug:
     
    export SPECPROD='bgs-gama'
    export  PIXPROD='bgs-gama'
    
    export DESI_SPECTRO_SIM=/global/cscratch1/sd/mjwilson/svdc-summer2018/spectro/sim
    export DESI_SPECTRO_REDUX=/global/cscratch1/sd/mjwilson/svdc-summer2018/spectro/redux
    
    wrap-fastframe --mpi --outdir /global/cscratch1/sd/mjwilson/svdc-summer2018/spectro/fframe/bgs-gama/    


    Example dir. structure (hand created):
    
    /global/cscratch1/sd/mjwilson/svdc-summer2018/spectro/sim/bgs-gama/20191201/00000002/


    Note:  skips arcs, processes flats and science. 
            --cframe for generating calibrated spectra directly. 

    See: 
    
    https://github.com/desihub/desisim/blob/master/bin/wrap-fastframe
    https://desisim.readthedocs.io/en/latest/_modules/desisim/scripts/fastframe.html
    
    
    7.4 minutes for 0 arc, 1 flat, 1 science exposures;  Success!
    '''
    
    if is_fastframe_done(explist, reduxdir, verbose=True):
        print('fastframe already done; skipping')
        
    else:
        logfilename = os.path.join(reduxdir, 'exposures', 'fastframe.log')
        
        os.makedirs(os.path.dirname(logfilename), exist_ok=True)

        print('Running fastframe batch job; should take ~7 min.')
        print('Starting at {}'.format(time.asctime()))
        print('Logging to {}'.format(logfilename))
        
        nodes   = 15

        cmd     = "srun -A desi -N {nodes} -n {nodes} -c 32 -C haswell -t 00:120:00 --qos interactive".format(nodes=nodes)
        cmd    += " wrap-fastframe --mpi --clobber"

        with open(logfilename, 'w') as logfile:
            err = subprocess.call(cmd.split(), stdout=logfile, stderr=logfile)
            
            if err != 0:
                print('ERROR {} running wrap-fastframe; see {}'.format(err, logfilename))
            
            else:
                print('Done at {}'.format(time.asctime()))

        if is_fastframe_done(explist, reduxdir, verbose=True):
            print('SUCCESS')
        
        else:
            print('ERROR; see {}'.format(logfilename))

In [ ]:
run_fastframe()

In [ ]:
#- Check individual framefile outputs
ntot = 0
nbad = 0

for night, expid, flavor in explist['NIGHT', 'EXPID', 'FLAVOR']:
    if flavor != 'flat' and flavor != 'science':
        continue

    for channel in ['b', 'r', 'z']:
        for spectrograph in range(10):
            camera    = channel + str(spectrograph)
            framefile = desispec.io.findfile('frame', night, expid, camera)
            ntot     += 1
            
            if not os.path.exists(framefile):
                nbad += 1
                print('Missing {} frame {}'.format(flavor, framefile))

if nbad > 0:
    print('Missing {}/{} frame files'.format(nbad, ntot))

else:
    print('All {} science and flat frame files generated'.format(ntot))

## Run the spectro pipeline
### First, create the production database.

In [ ]:
print(os.getenv('DESI_SPECTRO_DATA'))
print(os.getenv('DESI_SPECTRO_REDUX'))
print(os.getenv('SPECPROD'))

In [ ]:
from desispec.io import get_pipe_database


pipedbfile = get_pipe_database()

if not os.path.exists(pipedbfile):
    cmd = "desi_pipe create --db-postgres --force"
    cmd += " --data {}".format(os.getenv('DESI_SPECTRO_DATA'))
    cmd += " --redux {}".format(os.getenv('DESI_SPECTRO_REDUX'))
    cmd += " --prod {}".format(os.getenv('SPECPROD'))
    
    print(cmd)
    
    err = subprocess.call(cmd.split())
    
    assert err == 0
    
    ##  '/global/cscratch1/sd/mjwilson/svdc-summer2018/spectro/redux/bgs-gama/desi.db'
    assert os.path.exists(desispec.io.get_pipe_database())
    
    print('SUCCESS')
    
else:
    print('spectro pipeline DB file (%s) already exists; skipping' % pipedbfile)
    
##  Set environment variable for product. 
os.environ['DESI_SPECTRO_DB'] = os.getenv('DESI_SPECTRO_REDUX') + '/' + os.getenv('SPECPROD') + '/desi.db'

print('\n\nSet $DESI_SPECTRO_DB to:  ' + os.getenv('DESI_SPECTRO_DB'))

In [ ]:
pipe_setup_file = desispec.io.specprod_root() + '/setup.sh'

with open(pipe_setup_file) as fx:
    for line in fx:
        line = line.strip()
        if line.startswith('export '):
            keyvalue = line.split(' ')[1]
            key, value = keyvalue.split('=', maxsplit=1)
            if key in ('DESI_SPECTRO_REDUX', 'SPECPROD'):
                continue
            if key in os.environ and value != os.environ[key]:
                print('{} {} -> {}'.format(key, os.environ[key], value))
            elif key not in os.environ:
                print('Setting {}={}'.format(key, value))
            
            os.environ[key] = value

### Sync up with the actual files on disk

We didn't start with raw data files, so we'll skip over extraction and PSF-fitting steps.
`desi_pipe sync` will update the database from what files are actually on disk.

In [ ]:
err = subprocess.call('desi_pipe sync'.split())

assert err == 0

output = subprocess.check_output('desi_pipe top --once'.split())

print(output.decode())

## Notes
Preproc has fibermap and rawdata dependencies.  Currently, nothing happens as 
the state of rawdata is ready, rather than done. 

Cheap hack that would fix this:                                                                             cur.execute("UPDATE {tn} SET {cn}=3".format(tn='rawdata', cn='state'))

### Run pipeline scripts as a series of interactive jobs

`desi_pipe chain` would be a more convenient way of doing this,
but for the minitest it takes too long to wait for N>>1 jobs in the debug queue.


Ensure exposures are in:

DESI_SPECTRO_REDUX/SPECPROD/exposures/night/

as per get_exposures of 

https://github.com/desihub/desispec/blob/012070d7121a6c6c74ea51958667d49feb8bfa02/py/desispec/io/meta.py

In [ ]:
from desispec.pipeline import load_db


taskdir   = os.path.join(desispec.io.get_pipe_rundir(), 'minitest')

os.makedirs(taskdir, exist_ok=True)

dbpath    = desispec.io.get_pipe_database()
db        = load_db(dbpath, mode="w")

tasktypes = ['fiberflat', 'fiberflatnight', 'sky', 'starfit', 'fluxcalib', 'cframe', 'spectra', 'redshift']


print('Loading database:  %s' % dbpath)

for tasktype in tasktypes:
    for night in np.unique(explist['NIGHT']):
        print('Getting ready for night: %s' % night)
        
        ##  /global/common/software/desi/cori/desiconda/20180709-1.2.6-spec/code/desispec/0.27.0/lib/python3.6/site-packages/desispec-0.27.0-py3.6.egg/desispec/pipeline/db.py
        db.getready(night)

    taskfile = "{}/{}.tasks".format(taskdir, tasktype)

    cmd  = "desi_pipe tasks --tasktype {} --states ready,waiting ".format(tasktype)
    cmd += " > {}".format(taskfile)
    
    print('\n' + cmd)
    
    try:
        subprocess.check_call(cmd, shell=True)
    
    except subprocess.CalledProcessError:
        print('FAILED: {}'.format(cmd))
    
        break

    task_count = db.count_task_states(tasktype)

    print()
    print(task_count)

    if tasktype == 'redshift':
        ranks_per_task = 32
        cores_per_rank = 2
        n              = task_count['ready'] * ranks_per_task // 2  #- two iterations
        nodes          = (n-1) // 32 + 1
        runtime        = 59    #- minutes
    
    elif tasktype == 'spectra':
        ranks_per_task = 1
        cores_per_rank = 8
        n              = task_count['ready'] * ranks_per_task
        runtime        = 20    #- minutes
    
    else:
        ranks_per_task = 1
        cores_per_rank = 2
        n              = task_count['ready'] * ranks_per_task
        runtime        = 15    #- minutes

    nodes = (n*cores_per_rank-1) // 64 + 1

    if n > 0:
        t0 = time.time()

        cmd = 'srun -A desi -t {}:00 -C haswell --qos interactive'.format(runtime)
        cmd += ' -N {nodes} -n {procs} -c {cores} '.format(nodes=nodes, procs=n, cores=cores_per_rank)
        cmd += ' desi_pipe_exec_mpi --tasktype {} --taskfile {}'.format(tasktype, taskfile)
        
        logfilename = '{}/{}.log'.format(taskdir, tasktype)
        
        print('Running {} {} tasks'.format(n, tasktype))
        print('Logging to {}'.format(logfilename))
        print(cmd)
        
        with open(logfilename, 'w') as logfile:
            err = subprocess.call(cmd.split(), stdout=logfile, stderr=logfile)
        
            if err != 0:
                print('    ERROR {} for tasktype {}'.format(err, tasktype))
                print('    See {}'.format(logfilename))
            
            else:
                dt = time.time() - t0
                print('  DONE at {}'.format(time.asctime()))
                print('  {} took {:.1f} min'.format(tasktype, dt/60))
    
    elif task_count['waiting'] == 0 and task_count['done'] > 0:
        print('All {} tasks already run'.format(tasktype))
    
    else:
        print('No {} tasks ready to run; skipping'.format(tasktype))

for night in np.unique(explist['NIGHT']):
    db.getready(night)

print(subprocess.check_output('desi_pipe top --once'.split()).decode())